In [ ]:
# Get the system environment variables
import copy
import os
import pickle
import sys

# General Imports
import numpy as np
from numpy import linalg
from time import time
from IPython import display

# Variational
from auxiliary import collect_obs
from auxiliary import initialize_Ab0
from core import smoothing

# Plotting
from matplotlib import cm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Dynamic
from dynamics import sys_lorenz_96_ut as dynamics

%matplotlib inline

# Change the default figure size.
plt.rcParams['figure.figsize'] = (10.0, 8.0)

In [ ]:
def get_sample_path_file_name(base_dir):
    return '{}/sample.pickle'.format(base_dir)

In [ ]:
def get_result_file_name(base_dir):
    return '{}/result.pickle'.format(base_dir)

In [ ]:
def generate_sample_path(base_dir):
    '''
    TIME-WINDOW PARAMETERS:
    '''
    # Initial, final and time step.
    t0 = 0 
    tf = 4
    dt = 0.01
    
    # Define the time-window of inference.
    Tw = np.arange(t0, tf + dt, dt)

    # Number of discretized points.
    N = Tw.shape[0]

    '''
    SYSTEM SPECIFIC PARAMETERS:
    '''
    # Dimensionality of the system.
    D = 40

    # Stochastic Noise (variance).
    sigma_Noise = 4 * np.eye(D)

    # Drift parameters (sigma, rho, beta).
    theta_Drift = 8

    # Observation Noise (variance).
    obs_Noise = 1 * np.eye(D)

    # Define the observation density (# of observations per time unit).
    n_Obs = 8

    # We need at least one observation (per time unit)
    n_Obs = np.max([n_Obs, 1])

    # Observation operator: np.eye(D)
    H = np.eye(D)
    
    # Create the (artificial) true trajectory.
    xt_true = dynamics.system_path(Tw, D, sigma_Noise, theta_Drift)

    # Sample the noisy observations from the true path.
    obsX, obsY = collect_obs.collect_obs(xt_true, Tw, n_Obs, obs_Noise)
    
    data = {
        't0': t0,
        'tf': tf,
        'dt': dt,
        'Tw': Tw,
        'N': N,
        'D': D,
        'sigma_Noise': sigma_Noise,
        'theta_Drift': theta_Drift,
        'obs_Noise': obs_Noise,
        'n_Obs': n_Obs,
        'H': H,
        'xt_true': xt_true,
        'obsX': obsX,
        'obsY': obsY
    }
    
    file_name = get_sample_path_file_name(base_dir)
    
    with open(file_name, 'wb') as file:
        pickle.dump(data, file)
        
    return data

In [ ]:
def load_sample_path(base_dir):
    file_name = get_sample_path_file_name(base_dir)
    with open(file_name, 'rb') as file:
        return pickle.load(file)

In [ ]:
def run_smoothing(data, theta_Drift, sigma_Noise):
    # Define the time-window of inference.
    Tw = data['Tw']
    
    D = data['D']
    
    # Prior moment of initial condition noise variance.
    # p(x0) ~ N(mu, tau0)
    tau0 = 0.5 * np.eye(D)

    # Get the true sample value at time t=0
    prior_x0 = {'mu0': data['xt_true'][0], 'tau0': tau0}
    
    # Initial mean m(t=0)
    m0 = data['xt_true'][0] + 0.1 * np.random.randn(1)

    # Initial covariance matrix S(t=0): K*np.eye(D)
    S0 = 0.25 * np.eye(D)
    
    # ODE solver: {'Euler', 'Heun', 'RK2', 'RK4'}
    ode_method = 'Euler'
    
    sde_struct = {
        'Sig': sigma_Noise,
        'theta': theta_Drift,
        'Rig': data['obs_Noise'],
        'D': D, 
        'H': data['H'], 
        'obsX': data['obsX'],
        'obsY': data['obsY'],
        'px0': prior_x0,
        'Tw': Tw,
        'dt': data['dt'], 
        'N': data['N'], 
        'ode_method': ode_method,
        'checkGradf': False    
    }
    
    # Maximum number of iterations.
    nit = 500

    # Generate initial variational parameters (initial search point).
    Ab0 = initialize_Ab0.initialize_Ab0(S0, sde_struct)

    # Main Operation.
    print('[VGPA] (Smoothing) Experiment in progress. Please wait ...')

    # Full Variational approximation.
    Fmin, mParam = smoothing.smoothing(dynamics.energy_mode, Ab0, m0, S0, sde_struct, nit)    

    return Fmin, mParam, sde_struct

In [ ]:
def estimate_paramters(base_dir, theta_Drift, sigma_Noise, rate=0.01, nit=500):
    data = load_sample_path(base_dir)
    
    costs = []
    gradients = []
    changes = []
    
    # Inner loop
    Fmin, mParam, sde_struct = run_smoothing(data, theta_Drift, sigma_Noise)
    costs.append(Fmin)

    display.clear_output()
    print('Costs:')
    print(', '.join(['{:.2f}'.format(cost) for cost in costs]))
    print()
    
    it = 0
    
    Fmin_old = Fmin
    theta_Drift_old = copy.deepcopy(theta_Drift)
    
    while it < nit:
        Esde, Efx, Edf, dEsde_dm, dEsde_dS, dEsde_dth, dEsde_dSig = \
            dynamics.energy_mode(mParam['At'], mParam['bt'], mParam['mt'], mParam['St'], sde_struct)        
        
        if not np.any(dEsde_dth):
            break
    
        # Update parameter values        
        theta_Drift -= rate * dEsde_dth    

        # Inner loop
        tic = time()

        Fmin, mParam, sde_struct = run_smoothing(data, theta_Drift, sigma_Noise)
        
        ttime = time() - tic

        print('Runtime: {}'.format(ttime))
        print()
        
        display.clear_output()    
        
        Fmin_diff = abs(Fmin - Fmin_old)
        theta_Drift_diff = linalg.norm(theta_Drift - theta_Drift_old, 2)
        
        if Fmin_diff <= 1e-6 or theta_Drift_diff <= 1e-6:            
            break

        costs.append(Fmin)
        gradients.append(dEsde_dth)
        changes.append([Fmin_diff, theta_Drift_diff])
        
        print('Costs:')
        print(', '.join(['{:.2f}'.format(cost) for cost in costs]))
        print()
        
        print('Gradient:')
        print(', '.join(['{:.2f}'.format(item) for item in dEsde_dth])) 
        print()
        
        print('Changes:')
        print('{:.2f}, {:.2f}'.format(Fmin_diff, theta_Drift_diff))
        print()
        
        print('Parameter:')
        print(', '.join(['{:.2f}'.format(item) for item in theta_Drift])) 
        print()        
        
        # Increament iteration count
        it += 1
                
    return theta_Drift, sigma_Noise, Fmin, mParam, costs, gradients, changes, it

In [ ]:
if len(sys.argv) < 2:
    base_dir = './results/parameter-lorenz-96-ut/'
else:
    base_dir = './results/parameter-lorenz-96-ut/{}'.format(sys.argv[1])    

if not os.path.exists(base_dir):
    os.makedirs(base_dir)
        
if not os.path.exists(get_sample_path_file_name(base_dir)):
    print('Generating new sample path')
    generate_sample_path(base_dir)
    
D = 40
theta_Drift = 0
sigma_Noise= 4 * np.eye(D)

tic = time()

theta_Drift, sigma_Noise, Fmin, mParam, costs, gradients, changes, it = \
    estimate_paramters(base_dir, theta_Drift, sigma_Noise, rate=0.03)
    
ttime = time() - tic

with open(get_result_file_name(base_dir, 'wb')) as file:
    pickle.dump({
        'theta_Drift': theta_Drift,
        'sigma_Noise': sigma_Noise,
        'Fmin': Fmin,
        'mParam': mParam,
        'costs': costs,
        'gradients': gradients,
        'changes': changes,
        'it': it,
        'time': ttime
    }, file)